In [1]:
import torch
import numpy as np
import csv
import pandas as pd
from tc_learn import *

model = Net()

checkpoint = torch.load('./model/DNN_model.pt')
model.load_state_dict(checkpoint['model'])

# str = torch.Tensor([-0.332102407,2.325789922,0.620879471,0.336402609,-0.042428026,-0.036377326,-0.033060542])
# result = model(str)
# print(result)

<All keys matched successfully>

In [2]:
movie_path = "./data/bit_data_zscore_tier.csv"
movieq_numpy = pd.read_csv(movie_path, delimiter=",", encoding= 'cp949')
col_list = next(csv.reader(open(movie_path), delimiter=','))
mov = movieq_numpy[['genre','rating','country','score','company','writer','director','hit rate','zscore','robust','minmax','tier']]
movieq = torch.from_numpy(mov.values).float()
torch.manual_seed(1)

In [10]:
data = movieq[:,:-5]
target = movieq[:,-1]

predict = model(data)
for i in range(5416):
    predict[i] = round(predict[i].item())

predict.squeeze_(1)

correct = torch.zeros(5416,dtype=torch.bool)
for i in range(5416):
    if predict[i] == target[i]:
        correct[i] = True

sum = correct.sum(dim=0)
sum/5416

tensor(0.1475)

In [11]:
W = torch.Tensor([[1.1432, 0.9258, 0.8426, 0.8354, 0.8311],
                     [0.9211, 0.9496, 0.9636, 1.0065, 1.1431],
                      [1.0667, 1.6569, 0.9356, 0.4278, 0.3309],
                      [0.9891, 1.0194, 1.0133, 0.9847, 0.9710],
                      [1.8983, 0.1123, 0.1053, 1.8552, 0.3555],
                      [1.8921, 1.8494, 0.1108, 0.1072, 0.1060],
                      [1.8987, 0.1117, 0.1043, 0.9887, 0.0963]])
b = torch.Tensor([0.3324])

hypothesis = F.softmax(data.matmul(W) + b, dim=1)

In [12]:
target_one_hot = torch.zeros(5416,5,dtype=torch.float)

for i in range(5416):
    target_one_hot[i][int(target[i])-1] = 1

In [19]:
correct = torch.ones(5416,dtype=torch.bool)

for i in range(5416):
    maxval = max(hypothesis[i])
    for j in range(5):
        if hypothesis[i][j] == maxval:
            hypothesis[i][j] = 1
        else:
            hypothesis[i][j] = 0

for i in range(5416):
    for j in range(5):
        if target_one_hot[i][j] != hypothesis[i][j]:
            correct[i] = False
            break

sum = correct.sum(0)
sum/5416

tensor(0.3098)